<a href="https://colab.research.google.com/github/efstathios-chatzikyriakidis/energy-prices/blob/main/FisikonCNGPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tabula-py

!pip install --upgrade "kaleido==0.1.*"

In [ ]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

from datetime import datetime

from tabula import read_pdf

import plotly.express as px

import plotly.io as pio

In [ ]:
def is_date(date: str) -> bool:
  try:
    datetime.strptime(date, '%d/%m/%Y')

    return True
  except ValueError:
    return False

In [ ]:
req = requests.get("https://www.fisikon.gr/giati-fisikon")

soup = BeautifulSoup(req.text, "lxml")

In [ ]:
link = soup.find('a', text="εδώ")

In [ ]:
tables = read_pdf(link['href'], pages='all', pandas_options={'header': None})

In [ ]:
dfs = []

for table in tables:
  dfs.append(pd.DataFrame(table))

In [ ]:
for df in dfs:
  df.columns = ['date', 'price']

In [ ]:
all_data_df = pd.concat(dfs, ignore_index=True)

In [ ]:
all_data_df = all_data_df[(all_data_df['price'].apply(lambda o: type(o) == float)) & ~(all_data_df['price'].isnull()) & ~(all_data_df['date'].isnull())]

all_data_df = all_data_df[all_data_df['date'].apply(is_date)].copy()

all_data_df.set_index('date', inplace=True)

all_data_df.index = pd.DatetimeIndex(all_data_df.index)

all_data_df.sort_index(ascending=True, inplace=True)

In [ ]:
all_data_imputed_df = all_data_df.resample('D').min().ffill()

In [ ]:
all_data_imputed_df

In [ ]:
fig = px.line(all_data_imputed_df, y="price", labels={ "date": "Date", "price": "Price (€)" }, title=f"Fisikon CNG Price ({min(all_data_imputed_df.index).strftime('%d/%m/%Y')} - {max(all_data_imputed_df.index).strftime('%d/%m/%Y')})")

fig.update_xaxes(tickangle = 90)

fig.update_xaxes(ticks="outside", tickwidth=2, tickcolor='black', ticklen=10)
fig.update_yaxes(ticks="outside", tickwidth=2, tickcolor='black', ticklen=10)

fig.update_yaxes(nticks=20)
fig.update_xaxes(nticks=40)

fig

In [ ]:
pio.write_image(fig, "cng.png", width=8 * 300, height=2 * 300, scale=5) # 300 DPI